In [1]:
#!/usr/bin/env python3

import byte
import chardet
import csv
import json
import nilearn
import numpy as np
import os
import pandas as pd
import random
import regex as re
import string
import struct
import sys

from chardet import UniversalDetector as udet
from io import StringIO
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os import listdir as ls
from os.path import join as pjoin
from pandas import DataFrame as df
from string import printable
from typing import Union
from zipfile import ZipFile
from tqdm import tqdm
from unidecode import unidecode

from cimaq_utils import get_cimaq_dir_paths
from sniffbytes import flatten
from sniffbytes import loadfiles
from sniffbytes import loadimages
from cimaq_utils import repair_enc_task
from cimaq_utils import fetch_cimaq

from json_read import json_read

import sniffbytes as snif


from get_zip_contents import scanzip
from get_zip_contents import getnametuple

from removeEmptyFolders import removeEmptyFolders
from multiple_replace import multiple_replace

cimaq_dir = '~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data'
zeprimes = pjoin(cimaq_dir, 'task_files/zipped_eprime')


In [2]:
# import chardet
# import csv
# import json
# import nilearn
# import numpy as np
# import os
# import pandas as pd
# import random
# import regex as re
# import string
# import struct
# import sys

# from chardet import UniversalDetector as udet
# from io import StringIO
# from os.path import basename as bname
# from os.path import dirname as dname
# from os.path import expanduser as xpu
# from os import listdir as ls
# from os.path import join as pjoin
# from pandas import DataFrame as df
# from string import printable
# from typing import Union
# from zipfile import ZipFile
# from tqdm import tqdm
# from unidecode import unidecode

# import sniffbytes as snif

# def scanzip(archv_path: Union[os.PathLike, str],
#                ntpl: Union[str, list, tuple] = [],
#                exclude: Union[str, list, tuple] = []) -> object:


#     myzip = ZipFile(archv_path)
#     ntpl = [ntpl if ntpl else getnametuple(myzip)][0]

#     vals = df(
#             tuple(
#                 dict(zip(snif.evenodd(itm)[0], snif.evenodd(itm)[1]))
#                 for itm in tuple(
#                     tuple(
#                         snif.is_printable(repr(itm.lower()))
#                         .strip()
#                         .replace("'", "")
#                         .replace("'", "")
#                         .replace("=", " ")[:-2]
#                         .split()
#                     )[1:]
#                     for itm in tqdm(set(
#                         repr(myzip.getinfo(itm))
#                         .strip(" ")
#                         .replace(itm, itm.replace(" ", "_"))
#                         if " " in itm
#                         else repr(myzip.getinfo(itm)).strip(" ")
#                         for itm in ntpl
#                     ), desc = 'scanning archive')
#                 )
#             ),
#             dtype="object",
#         ).sort_values("filename").reset_index(drop=True)

#     vals['src_names'] = sorted(ntpl)
#     vals['bsheets'] = [myzip.open(row[1].src_names).read()
#                        for row in vals.iterrows()]
#     myzip.close()
#     sniffed = df(snif.scan_bytes(row[1].bsheets)
#                for row in vals.iterrows()).to_dict()
#     return df.from_dict({**vals.to_dict(), **sniffed}, orient = 'index')



In [3]:
# cimaq_infos = df(tuple(scanzip(apath) for apath in
#                        tqdm(loadimages(xpu(zeprimes)),
#                             desc = 'sniffing')))
cimaq_infos = pd.concat(val.T for val in
                        df(tuple(scanzip(apath) for apath in
                                 tqdm(loadimages(xpu(zeprimes)),
                                      desc = 'sniffing')),
                          dtype = object)[0].values.flatten()).dropna()

cimaq_infos['newsheets'] = [snif.mkfrombytes(
                                row[1].bsheets,
                                encoding = row[1]['encoding'],
                                delimiter = row[1]['delimiter'],
                                hdr = row[1]['has_header'],
                                dup_index = row[1]['dup_index']).decode(
                                    'utf8', 'replace').replace(
                                        '�', '').encode().decode('utf').strip()
                            
                            
                             for row in tqdm(cimaq_infos.iterrows(),
                                             desc = 'repairing')]

cimaq_infos['as_df'] = [df((line.split('\t') for line in
                             unidecode(row[1].newsheets).split('\\n')),
                          dtype = object).T.set_index(0, drop = True).T
                         if row[1].has_header else
                         df((line.split('\t') for line in
                             unidecode(row[1].newsheets).split('\\n')),
                           dtype = object)
                         for row in tqdm(cimaq_infos.iterrows(),
                                         desc = 'creating Pandas DataFrames')]






scanning archive: 100%|██████████| 9/9 [00:00<00:00, 5121.95it/s]

scanning archive: 100%|██████████| 9/9 [00:00<00:00, 6175.16it/s]

scanning archive: 100%|██████████| 3/3 [00:00<00:00, 4917.12it/s]

scanning archive: 100%|██████████| 10/10 [00:00<00:00, 5674.88it/s]

scanning archive: 100%|██████████| 9/9 [00:00<00:00, 5570.95it/s]

scanning archive: 100%|██████████| 13/13 [00:00<00:00, 5516.03it/s]

scanning archive: 100%|██████████| 9/9 [00:00<00:00, 4432.16it/s]

scanning archive: 100%|██████████| 9/9 [00:00<00:00, 5237.06it/s]

scanning archive: 100%|██████████| 9/9 [00:00<00:00, 6623.75it/s]

scanning archive: 100%|██████████| 9/9 [00:00<00:00, 5416.66it/s]

scanning archive: 100%|██████████| 13/13 [00:00<00:00, 6675.56it/s]

scanning archive: 100%|██████████| 10/10 [00:00<00:00, 5722.11it/s]

scanning archive: 100%|██████████| 10/10 [00:00<00:00, 6559.75it/s]

scanning archive: 100%|██████████| 9/9 [00:00<00:00, 5603.20it/s]

scanning archive: 100%|██████████| 10/10 [00:00<00:0

scanning archive: 100%|██████████| 10/10 [00:00<00:00, 5744.83it/s]

scanning archive: 100%|██████████| 13/13 [00:00<00:00, 6605.20it/s]

scanning archive: 100%|██████████| 9/9 [00:00<00:00, 5491.52it/s]

scanning archive: 100%|██████████| 10/10 [00:00<00:00, 5629.94it/s]

scanning archive: 100%|██████████| 9/9 [00:00<00:00, 5375.02it/s]

scanning archive: 100%|██████████| 10/10 [00:00<00:00, 5602.12it/s]

scanning archive: 100%|██████████| 11/11 [00:00<00:00, 5566.76it/s]

scanning archive: 100%|██████████| 10/10 [00:00<00:00, 5418.30it/s]

scanning archive: 100%|██████████| 9/9 [00:00<00:00, 6470.47it/s]

sniffing: 100%|██████████| 103/103 [00:09<00:00, 11.30it/s]
/home/francois/.local/lib/python3.8/site-packages/pandas/core/internals/construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=o

In [4]:
cimaq_infos.iloc[150]['as_df']
# with open(pjoin(os.getcwd(), 'test.txt'), 'wb') as newfile:
#     newfile.write(cimaq_infos.iloc[124]['bsheets'])
#     newfile.close()

,0,1,2,3,4,5,6,7,8
0,21,Enc,Enc14,Old22,Encoding,113.022,3,Fixation,116.022
1,39,Enc,Enc28,Old24,Encoding,235.011,3,Fixation,238.011
2,58,Enc,Enc39,Old58,Encoding,350.501,3,Fixation,353.5
3,59,Enc,Enc40,Old60,Encoding,356.5,3,Fixation,359.5


In [30]:
# cimaq_infos['newsheets'] = [snif.mkfrombytes(
#                                 row[1].bsheets,
#                                 encoding = row[1]['encoding'],
#                                 delimiter = row[1]['delimiter'],
#                                 hdr = row[1]['has_header'],
#                                 dup_index = row[1]['dup_index']).decode(
#                                     'utf8', 'replace').replace(
#                                         '�', '').encode().decode().strip()
                            
                            
#                              for row in tqdm(cimaq_infos.iterrows(),
#                                              desc = 'repairing')]

cimaq_infos['as_df'] = [df((line.split('\t') for line in
                             snif.mkfrombytes(
                                row[1].bsheets,
                                encoding = row[1]['encoding'],
                                delimiter = row[1]['delimiter'],
                                hdr = row[1]['has_header'],
                                dup_index = row[1]['dup_index']).decode(
                                    'utf8', 'replace').replace(
                                        '�', '').encode().decode().strip().split('\\n'
                                            ))).T.set_index(0, drop = True).T
                         if row[1].has_header else
                         df((line.split('\t') for line in
                             snif.mkfrombytes(
                                row[1].bsheets,
                                encoding = row[1]['encoding'],
                                delimiter = row[1]['delimiter'],
                                hdr = row[1]['has_header'],
                                dup_index = row[1]['dup_index']).decode(
                                    'utf8', 'replace').replace(
                                        '�', '').encode().decode().strip().split('\\n')))
                         for row in tqdm(cimaq_infos.iterrows(),
                                         desc = 'creating Pandas DataFrames')]

creating Pandas DataFrames: 683it [01:06, 10.22it/s]


In [31]:
cimaq_infos.iloc[667]['as_df']





,TrialNumber,Category,TrialCode,OldNumber,CorrectSource,Stim_RESP,Stim_RT,Stim_ACC\n1,Enc,Enc00,...,3,1949,0\n120,Enc,Enc78,Old20,8,3,1634,0


In [32]:
cimaq_infos2 = pd.concat(val.T for val in cimaq_infos[0].values.flatten())
cimaq_infos2 = cimaq_infos2.dropna()
cimaq_infos2['newsheets'] = [snif.mkfrombytes(
                                row[1].bsheets,
                                encoding = row[1]['encoding'],
                                delimiter = row[1]['delimiter'],
                                hdr = row[1]['has_header'],
                                dup_index = row[1]['dup_index']).decode(
                                    'utf8', 'replace').replace('�', '').encode()
                             for row in tqdm(cimaq_infos2.iterrows(),
                                             desc = 'repairing')]

cimaq_infos2['as_df'] = [df((line.split('\t') for line in
                             snif.mkfrombytes(
                                row[1].bsheets,
                                encoding = row[1]['encoding'],
                                delimiter = row[1]['delimiter'],
                                hdr = row[1]['has_header'],
                                dup_index = row[1]['dup_index']).decode(
                                    'utf8', 'replace').replace('�', '').encode().decode().strip().split(
                                 '\\n'))).T.set_index(0, drop = True).T
                         if row[1].has_header else
                         df((line.split('\t') for line in
                             snif.mkfrombytes(
                                row[1].bsheets,
                                encoding = row[1]['encoding'],
                                delimiter = row[1]['delimiter'],
                                hdr = row[1]['has_header'],
                                dup_index = row[1]['dup_index']).decode(
                                    'utf8', 'replace').replace('�', '').encode().decode().strip().split(
                                 '\\n')))
                         for row in tqdm(cimaq_infos2.iterrows(),
                                         desc = 'creating Pandas DataFrames')]

KeyError: 0

In [ ]:
cimaq_infos2['as_df'].sample(1).values[0]

In [ ]:
cimaq_infos2.iloc[99].as_df

In [ ]:
cimaq_infos2['to_csv'] = [pd.read_csv(StringIO(row[1].unidecoded),
                                      sep = '\t',
                                      header = [0 if row[1].has_header
                                                else None][0],
                                      engine = 'c')
#                                       quoting = 1)
                              for row in cimaq_infos2.iterrows()]

In [ ]:
cimaq_infos2.iloc[667].to_csv

In [ ]:
cimaq_infos2['cleaned'] = ['\n'.join(['\t'.join(line.split()) for line in
                                      snif.is_printable(row[1].newsheets.decode(
                                          row[1].encoding).encode(
                                          'utf8', 'replace').decode()).splitlines()]).encode().decode()
                           for row in cimaq_infos2.iterrows()]
cimaq_infos2['cleaned']

In [ ]:
[pd.read_csv(StringIO(row[1].newsheets.decode(row[1].encoding)), sep = '\t')
 for row in cimaq_infos2.iterrows()][666]

In [ ]:
list(scanzip(loadimages(xpu(zeprimes))[0]).keys())

In [ ]:
chardet.detect(b'0xf8')#0xf
# chr(int.from_bytes(b'0xf', sys.byteorder))
# bytes(int.from_bytes(b'0xf8', sys.byteorder))
# import codecs
# help(codecs)
[chr(item) for item in list(b'0xf')]
[itm for itm in list]
int.from_bytes(b'0xf', sys.byteorder)

In [ ]:
def sniffzip(archv_path: Union[os.PathLike, str],
               ntpl: Union[str, list, tuple] = [],
               exclude: Union[str, list, tuple] = []) -> object:
    scan_bytes(myzip.open(row[1].))

In [ ]:
def get_zinfos(archv_path: Union[os.PathLike, str],
               ntpl: Union[str, list, tuple] = [],
               exclude: Union[str, list, tuple] = []) -> object:
        return pd.concat([df(snif.evenodd(repr(itm)[8:-1].strip().replace('=', ' ').split())).T[1]
                          for itm in
                (ZipFile(archv_path).__dict__['filelist'][1:],
                ZipFile(archv_path).close())[0]
                if '__MACOSX' not in repr(itm)], axis = 1).T

# atest = list(get_zinfos(loadimages(xpu(zeprimes))[0])[0].loc['NameToInfo'][0].values())
# btest = get_zinfos(loadimages(xpu(zeprimes))[0])[0].loc['filelist'][0]
# atest# json.dumps(repr(get_zinfos(loadimages(xpu(zeprimes))[0])[0].loc['NameToInfo'].values.tolist()).split())
# #.replace('=', '":').replace('\\', '')
get_zinfos(loadimages(xpu(zeprimes))[0])
df(repr(itm[1])[8:-1].replace('=', ' ').split() for itm in
      ZipFile(loadimages(xpu(zeprimes))[0]).__dict__['NameToInfo'].items()
      if '__MACOSX' not in itm)[1:]

In [ ]:
btest

In [ ]:
########## GOOD ONE ######################333
def get_zinfos(archv_path: Union[os.PathLike, str],
               ntpl: Union[str, list, tuple] = [],
               exclude: Union[str, list, tuple] = []) -> object:
    return ((df(zip(tuple(itm.strip("'") for itm in repr(
               ZipFile(archv_path).getinfo(nm))[8:-1].strip().replace(
                   b'='.decode(), ' ').split()#.replace(chr(34), '').split())
                   for nm in snif.filter_lst_exc(
                       exclude, [ntpl if ntpl
                       else getnametuple(ZipFile(archv_path))][0])))),
            ZipFile(archv_path).close())[0])

get_zinfos(loadimages(xpu(zeprimes))[0])[0]
#.replace('=', '":').replace('\\', '')



In [ ]:
def ls_archv(apath: Union[str, os.PathLike], ntpl: Union[str, list, tuple] = []) -> object:
    

In [ ]:
def get_zip_contents(
    archv_path: Union[os.PathLike, str],
    ntpl: Union[str, list, tuple] = [],
    exclude: Union[str, list, tuple] = [],
    withbytes: bool = False,
    to_sniff: bool = False,
    to_close: bool = True,
) -> object:
    myzip = ZipFile(archv_path)
    ntpl = snif.filter_list_exc(exclude,
                                [ntpl if ntpl else snif.getnametuple(myzip)][0])

    vals = (
        df(
            tuple(
                dict(zip(evenodd(itm)[0], evenodd(itm)[1]))
                for itm in tuple(
                    tuple(
                        force_ascii(repr(itm.lower()))
                        .strip()
                        .replace("'", "")
                        .replace("'", "")
                        .replace("=", " ")[:-2]
                        .split()
                    )[1:]
                    for itm in set(
                        repr(myzip.getinfo(itm))
                        .strip(" ")
                        .replace(itm, itm.replace(" ", "_"))
                        if " " in itm
                        else repr(myzip.getinfo(itm)).strip(" ")
                        for itm in ntpl
                    )
                )
            ),
            dtype="object",
        )
        .sort_values("filename")
        .reset_index(drop=True)
    )
    vals[["src_name", "ext"]] = [(nm, os.path.splitext(nm)[1]) for nm in ntpl]
    vals["filename"] = [
        "_".join(
            pd.Series(
                row[1].filename.lower().replace("/",
                                                "_").replace("-",
                                                             "_").split("_")
            ).unique()
            .__iter__()
        )
        for row in vals.iterrows()
    ]
    if exclude:
        vals = vals.drop(
            [
                row[0]
                for row in vals.iterrows()
                if row[1].filename
                not in filter_lst_exc(exclude, [itm.lower() for itm in vals.filename])
            ],
            axis=0,
        )

    if withbytes:
        vals["bsheets"] = [
#             snif.strip_null() 
            myzip.open(row[1].src_name).read().lower() for row in vals.iterrows()
        ]
        
    if to_sniff:
        vals[["encoding", "delimiter", "has_header", "width", "dup_index", "nrows"]] = \
            [tuple(snif.scan_bytes(row[1].bsheets).values()) for row in vals.iterrows()]
    if to_close:
        myzip.close()
        return vals
    else:
        return (myzip, vals)


In [ ]:
# azpath = loadfiles(loadimages(xpu(zeprimes))).fpaths[0]
# with zipfile.Zipfile(azpath) as myzip:
    


In [ ]:
test =  dict((bname(apath.replace('-', '_').strip()),
                    get_zip_contents(apath, withbytes = True,
                         to_sniff = True, to_close = True))
                    for apath in tqdm(loadimages(zeprimes),
                          desc = 'scan sniff'))
test


In [ ]:
# # from sniffbytes import no_ascii
# # chardet.detect(no_ascii('bon matin tin�tin!').encode())
# # Union[str, bytes]
# ''.join([chr(int.from_bytes(int.from_bytes(itm.encode(), sys.byteorder)))#.encode(encoding)
#  for itm in ])
# # ==  dict(json.dumps(set(string.printable)))
set([chr(int.from_bytes(itm, sys.byteorder)).encode('utf32')
 for itm in [itm.encode() for itm in list(string.printable)]])
[int.from_bytes(itm.encode(), sys.byteorder) for itm in list(string.printable)[-2:]]
# '\x00'.encode('ISO-8859-1')
encodings = ['Windows-1252', 'utf32', 'utf8', 'ISO-8859-1']
[int.from_bytes(itm.encode(encoding), sys.byteorder) for itm in list(string.printable)]
int.from_bytes(b'', sys.byteorder)
chardet.detect(b'\x00'), chardet.detect(b'None')
is_printable = [itm.encode('utf32') for itm in list(string.printable)], '\x00'.encode('ascii')

def force_encoding(inpt: bytes, encoding: str = None) -> bytes:
    """
    Source: https://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces-using-python
    """
    notnull_printable = set(itm.encode(encoding) for itm in list(string.printable)
                            if int.from_bytes(itm, sys.byteorder) != 0)
#                             if chr(int.from_bytes(itm, sys.byteorder)) != "\x00")
    encoding = [encoding if encoding else get_bencod(inpt)][0]
    return "".encode(encoding).join(filter(lambda x: x in notnull_printable, inpt))


# is_printable = , '\x00'.encode('ascii')
# '0xff'.encode('utf32') in is_printable
utf32p = ''.encode('utf32').join(set(itm.encode('utf32') for itm in list(string.printable)))
windows1252p = ''.encode('Windows-1252').join(set(itm.encode('Windows-1252') for itm in list(string.printable)))
utf8p = list(itm.encode('utf8') for itm in list(string.printable))
utf32p = list(itm.encode('utf32') for itm in list(string.printable))
test = set(itm.encode('utf32') for itm in list(string.printable)
           if int.from_bytes(itm.encode('utf32'),
                                 'little') != '\x00'.encode('utf32'))
# test == utf32set
# force_encoding(utf8p, 'Windows-1252')
# chr(0)
[int.from_bytes(itm, sys.byteorder) for itm in utf32set]

astring = "    Salut\tbébé,\tmon\tnom\tc'\x00est\sFrançois!"
chardet.detect(("Salut bébé, mon nom c'est François!").encode())
# weird = list(chr(itm) for itm in tuple(''.join(utf32set).encode()))[-5]
# astring.encode('ascii')

# clean_astring = ''.join(itm for itm in list(astring) if
#                         int.from_bytes(itm.encode('utf32'), sys.byteorder) != 0)

def clean_bytes(inpt: bytes, encoding: str = None) -> bytes:
    return ''.join(itm for itm in list(inpt) if
                        chr(int.from_bytes(itm)) != chr(0).encode(encoding) and itm in
                       set(itm.encode(encoding) for itm in list(string.printable)))
# chr(int.from_bytes(weird.encode('ascii'), sys.byteorder))
# list(astring)
# int.from_bytes('\\x00'.encode(), 'big')
# int.from_bytes('\\x00'.encode(), sys.byteorder) == int.from_bytes('\\x00'.encode(), 'little')
maybe_null == notnull_printable
(astring, clean_astring)
# '\x00'.encode('utf32')
nareps = [chr(0).encode('ascii'), chr(0).encode('utf32'), chr(0).encode('ISO-8859-1')]
[int.from_bytes(itm, sys.byteorder) for itm in nareps]


In [ ]:
astring = "    \\\tSalut\tbé bé,\tmon\tnom\tc'\x00est\sFrançois!\S"
bstring = astring.encode('utf16')
narep = chr(0).encode(snif.get_bencod(bstring))
chr(0).encode('utf16') == chr(0).encode(snif.get_bencod(bstring))
[chr(itm) for itm in list(bstring)]# chardet.detect('ç'.encode())
# [chr(itm) for itm in list(bstring) if chr(itm) != ]
narep, bstring, bstring.replace(narep, ''.encode(snif.get_bencod(bstring)))
tuple(zip(list(astring), list(bstring)))

In [ ]:
chr(int.from_bytes(''.encode(), sys.byteorder)).encode('utf32'), chr(int.from_bytes('0'.encode(), sys.byteorder)).encode('utf32')

In [ ]:
test

In [ ]:


# import bytes
# # import format as fmt

# def strip_null(inpt: bytes, nullrep: bytes = None, encoding: str = None):
#     encoding = [encoding if encoding else snif.get_bencod(inpt)][0]
#     nullrep = [nullrep if nullrep else snif.get_nullrep(inpt, encoding)][0]
#     return {
#         format(print("[%q]",
#                    bytes.Trim(byte(" !!! Achtung! Achtung! !!! "), "! ")))}


def strip_null(inpt: bytes, encoding: str = None) -> bytes:
    ''' Remove null bytes from byte stream with proper representation
        Adapted from:
        https://stackoverflow.com/questions/21017698/converting-int-to-bytes-in-python-3
        All files end by a null byte, so the last byte in a file shows
        how null bytes are represented within this file '''
    encoding = [encoding if encoding else get_bencod(inpt)][0]
#     return inpt.decode(encoding, 'replace').replace('�', '').replace(
#         ''.join(get_nullrep(inpt, encoding)), '').encode()
    try:
        return inpt.replace(chr(int.from_bytes(
                   b"\x00", sys.byteorder)).encode(encoding), ''.encode(encoding))
    except UnidecodeError:
        return inpt.replace(get_nullrep(inpt, encoding), ''.encode(encoding))

In [ ]:
testbytes = b'bonjour\xff\xfe\x00\x00\x00\x00\x00\x00'
chardet.detect(snif.strip_null(testbytes))#.encode('ISO-8859-1')
chardet.detect(testbytes)

In [ ]:
# def get_nullrep(inpt, encoding: str = None) -> bytes:
#     return [itm for itm in list(inpt) if
#             chr(int.from_bytes(itm, sys.byteorder)).encode(encoding) == "\x00".encode(encoding)]


In [ ]:
import codecs
help(codecs.lookup('utf8').streamreader)

In [ ]:
newtest = [dict((subject[0], 
                df.from_dict(dict((itm for itm in
                                   list(snif.scan_bytes(snif.force_utf8(
                  row[1].bsheets, row[1].encoding)).items()) + \
                      [('filename', row[1].filename),
                       (('pscid', 'dccid'), row[1].filename.split('_')[:2])])),
                             orient = 'index'))
               for row in tqdm(subject[1].iterrows(), desc = 'new sniff'))
           for subject in list(test.items())]

In [ ]:
newtest

In [ ]:
# newtest
# # both = {**test, **newtest}
# both = tuple({**itm[0], **itm[1]} for itm
#              in )
# both
# both = tuple(zip([itm.values() for itm in test], [itm.values() for itm in newtest]))
# both
tuple(zip(test.values(), newtest.values()))

In [ ]:
def nice_xtrct(archv_path: Union[os.PathLike, str],
    ntpl: Union[str, list, tuple] = None,
    to_xtrct: Union[str, list, tuple] = 'all',
    exclude: Union[str, list, tuple] = [],
    to_close: bool = True,
    withbytes: bool = False,
    dst_path: Union[os.PathLike, str] = None) -> object:
    dst_path = [dst_path if dst_path
                else pjoin(os.get_cwd(),
                           os.path.splitext(bname(archv_path))[0])][0]
    os.makedirs(dst_path, exist_ok=True)
    myzip = zipfile.ZipFile(archv_path)
    ntpl = filter_lst_exc([ntpl if ntpl else getnametuple(myzip)][0])
    contents = get_zip_contents(archv_path, ntpl, excllude, to_xtrct,
                                to_close = True, withbytes = True, to_sniff = True)
    xtrct_lst = vals.loc[
        [
            row[0]
            for row in vals.iterrows()
            if row[1].filename
            in filter_lst_inc(to_xtrct, list(vals.filename), sort=True)
        ]
    ]
    [
        shutil.move(
            myzip.extract(member=row[1].src_name, path=dst_path),
            pjoin(
                dst_path,
                "_".join(
                    pd.Series(
                        row[1].filename.lower().replace("-", "_").split("_")
                    ).unique()
                ),
            ),
        )
        for row in tqdm(xtrct_lst.iterrows(), desc="extracting")
    ]
    vals = vals.loc[
        [
            row[0]
            for row in vals.iterrows()
            if row[1].filename not in xtrct_lst.values
        ]
    ]
    removeEmptyFolders(dst_path, False)
    


In [ ]:
b_printables = [chr(int.from_bytes(itm.encode(), sys.byteorder)).encode() for itm in string.printable]
str(b_printables[-1]) == repr(b_printables[-1])

def force_to(inpt, src_enc: str = None, dst_enc: str = 'utf8'):
    inpt = get_bytes(inpt)
    as_ints = [[line.split()]]
    
    
chr(list(asheet)[0])
list(asheet).__len__()
# chr(int.from_bytes(b_printables[-1], sys.byteorder))

# [chr(itm) for itm in list(asheet)]
# help(chr)

In [ ]:
# cimaq_topdir = '~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_03-19/derivatives/'
# big = loadfiles([apath for apath in loadimages(xpu(cimaq_topdir))
#                  if os.path.isfile(apath)])
# big[['dccid', 'pscid']] = [([re.compile('\d{6}').search(row[1].fname).group()
#                              if re.compile('\d{6}').search(row[1].fname) != None
#                              else None][0],
#                             [re.compile('\d{7}').search(row[1].fname).group()
#                              if re.compile('\d{7}').search(row[1].fname) != None
#                              else None][0])
#                            for row in big.iterrows()]
'\\x0'.encode('utf16')
chr(int.from_bytes(b"\0xff", sys.byteorder))

In [ ]:
df(asheet)

# test2 = pd.concat([itm[1] for itm in tqdm(sorted(list(test)), desc = 'concatenate')])

# test2['forced_utf8'] = [snif.force_utf8(row[1].bsheets, row[1].encoding)
#                         for row in test2.iterrows()]

#.filename,
#                    snif.scan_bytes(row[1].forced_utf8, encoding='utf'))
#                         for row in itm[1].iterrows())
#              for itm in tqdm(list(test.items()),  desc = 'scan & sniff utf8')]


In [ ]:
list(test.items())[0]

In [ ]:
assert newtest.loc['encoding'].all() == 'ascii' or 'UTF-8'
chr(int.from_bytes('n\a'.encode(), sys.byteorder))

b'n\a' == 'n\a'.encode()

chr(int.from_bytes('n\a'.encode(), sys.byteorder)).encode('utf8')


In [ ]:
def last_item(inpt: bytes, encoding: str = None) -> bytes:
    ''' Returns null byte representation as bytes in native file encoding'''
    encoding = [encoding if encoding else snif.get_bencod(inpt)][0]
#     return bytes([inpt.splitlines(keepends = True)[-1][-1]]).decode(encoding).encode(encoding)
#     rep = chr(list(inpt)[-1]).encode(encoding)
    last1 = inpt.splitlines()[-1].split()[-1]
    last2 = [chr(itm).encode(encoding) for itm in
             list(chr(list(inpt.splitlines()[-1])[-1]).encode(encoding))]
    return (last1, last2)
#     return [chr(itm) for itm in list(chr(list(inpt.splitlines()[-1])[-1]).encode(encoding))]

In [ ]:
# list(chr(int.from_bytes(b'\x00', sys.byteorder)))
# snif.get_bencod(chr(0))
int.from_bytes(b'\x00', sys.byteorder)
aguy = random.sample(list(test.values()), 1)
aguy

In [ ]:
aguy = list(test.values())[0]
aguy['size_check'] = [int(row[1].file_size) == len(list(row[1].bsheets))
                      for row in aguy.iterrows()]
aguy['na_reps'] = [snif.get_nullrep(row[1].bsheets) for row in aguy.iterrows()]
aguy['last_item'] = [last_item(row[1].bsheets, row[1].encoding) for row in aguy.iterrows()]

aguy[['n_zbytes', 'n_scanbytes', 'chkup']] = [(int(len(list(row[1].bsheets))), int(row[1].file_size),
                                      (int(len(list(row[1].bsheets))) == int(row[1].file_size)))
                                     for row in aguy.iterrows()]
aguy

In [ ]:


aguy['utf8len'] = [len(row[1].forced_utf8.splitlines()) for row in aguy.iterrows()]
aguy['sheet_diff_tophalf'] = [row[1].bsheets.splitlines()[:int(np.floor(len(
                                 row[1].bsheets.splitlines())/2))] == \
                                 row[1].forced_utf8.splitlines()[int(np.floor(len(
                                     row[1].bsheets.splitlines())/2)):]
                              for row in aguy.iterrows()]
aguy['sheet_diff_bothalf'] = [[line.strip() for line in
                               row[1].bsheets.splitlines()[:int(np.floor(len(
                                 row[1].bsheets.splitlines())/2))]] == \
                                 [line.strip() for line in
                                  row[1].forced_utf8.splitlines()[int(np.floor(len(
                                     row[1].bsheets.splitlines())/2)):]]
                              for row in aguy.iterrows()]

aguy['nrows_test'] = [(len(row[1].bsheets.splitlines()) == len(row[1].forced_utf8.splitlines()))
                      for row in aguy.iterrows()]
aguy[['missing_line_index', 'missing_line_values']] = [(row[1].nrows_test)*2 if row[1].nrows_test else
                        sorted([(line[0], line[1]) for line in enumerate(row[1].bsheets.splitlines()) if
                                snif.force_utf8(line[1], 'utf8') not in row[1].forced_utf8.splitlines()])
                       for row in aguy.iterrows()]
# aguy['eq_lines'] = [len([line[0] for line in enumerate(tuple(zip(row[1].bsheets.splitlines(),
#                       row[1].forced_utf8.splitlines()))) if line[1][0] == line[1][1]])
#                     == len
#                     for row in aguy.iterrows()]

aguy.last_item.iloc[6][1][0].decode('utf16').encode('utf8').decode('utf8')
aguy.forced_utf8, aguy.bsheets
aguy['missing_line'].iloc[5]
# aguy.iloc[5].bsheets.splitlines(), aguy.iloc[5].forced_utf8.splitlines()

In [ ]:
asheet = aguy.iloc[4].bsheets[0]
utfsheet = aguy.iloc[4].forced_utf8[0]
asheet == utfsheet

In [ ]:
# asheet = aguy.sample(1).bsheets.values[0]
# last_item(asheet)


In [ ]:

enc = snif.get_bencod(abyte)
nonull = strip_null(abyte, enc)
nonull.splitllines()[:4], abyte.splitlines()[:4]
# aguy['no_null'] = [strip_null(row[1].bsheets,
#                               row[1].encoding)#.encode(row[1].encoding)
#                    for row in aguy.iterrows()]
# aguy['bprints'] = [snif.bytes_printable(row[1].bsheets).encode(snif.get_bencod(row[1].bsheets))
#                  for row in aguy.iterrows()]
# checkfx = []


# list(abyte.splitlines()[-1])
# [chr(itm).encode() for itm in
#  [list(row[1].bsheets.splitlines()[-1])[-1]
# for row in aguy.iterrows()]]

In [ ]:
inpt = list(test.values())[0].iloc[0].bsheets
encoding = list(test.values())[0].iloc[0].encoding

def strip_null(inpt: bytes, encoding: str = None) -> bytes:
    ''' Remove null bytes from byte stream with proper representation
        Adapted from:
        https://stackoverflow.com/questions/21017698/converting-int-to-bytes-in-python-3
        All files end by a null byte, so the last byte in a file shows
        how null bytes are represented within this file '''
    encoding = [encoding if encoding else snif.get_bencod(inpt)][0]
    return '\n'.join(inpt.decode(encoding, 'replace').replace('�', '').replace(
        ''.join(snif.get_nullrep(inpt, encoding)), '').splitlines())

pd.read_csv(StringIO(snif.mkfrombytes(inpt.strip()).decode()), sep = '\t')
#     narep =''.encode(encoding).join([itm.encode(encoding) for itm in
#                                            snif.get_nullrep(inpt, encoding)])
#     return inpt.replace(narep, '|'.encode(encoding))
#         inpt = inpt.replace(narep.encode(encoding),
#                             '|'.encode(encoding)).replace('|'.encode(encoding),
#                                                           ''.encode(encoding))
#         return inpt
#     return snif.bytes_printable(inpt.replace(,
#                                              repval).replace(chr(0).encode(encoding),
#                                         repval).replace(chr(1).encode(encoding),
#                                                         repval))
# snif.mkfrombytes(strip_null(inpt, 'utf8').encode())
# pd.read_csv(StringIO(snif.mkfrombytes(inpt).decode(encoding)),
#             sep='\t', header = [0 if snif.get_has_header(strip_null(inpt).encode(encoding)) else None][0])
# inpt.decode(snif.get_bencod(strip_null(inpt)), 'ignore').replace('�', '')
# pd.read_csv(StringIO(), sep='\t')
# [[list(itm for itm in line.split() if itm not in nareps)]
#  for line in snif.bytes_printable(inpt)]
# def get_na_reps(inpt: bytes, encoding: str = None) -> Union[list, int, bytes]:
#     inpt = snif.bytes_printable(inpt)
#     encoding = [encoding if encoding else snif.get_bencod(inpt)]
#     return [chr(itm).encode(encoding) for itm
#             in [chr(nulb) for nulb in
#                 list(snif.get_nullrep(snif.bytes_printable(inpt)))][0]]

# get_na_reps(inpt)
# # [list(''.encode(encoding).join((line.split())))
# #  for line in inpt.splitlines()]
# chr(int.from_bytes('0xf8', sys.byteorder))


In [ ]:
assert 3+5 ==9

In [ ]:
from multiple_replace import multiple_replace

def clear_nullbytes(inpt: bytes, encoding: str = None):
    inpt = snif.bytes_printable(inpt)
    encoding = snif.get_bencod(inpt)
    toclear = dict((itm.encode(encoding), '|'.encode(encoding))
                   for itm in
                   )


    for narep in nareps:
        inpt = inpt.replace(narep, '|'.encode(encoding))
    return inpt
#     return inpt.replace('|'.encode(encoding), 'nan'.encode(encoding))
# multiple_replace(toclear, snif.bytes_printable(abyte), encoding)
# help(str.replace)
# b_printable = ''.encode(encoding).join([ch.encode(encoding)
#                               for ch in list(string.printable)])
# b_printable

inpt = list(test.values())[0].iloc[8].bsheets
encoding = list(test.values())[0].iloc[8].encoding
# new = snif.bytes_printable(abyte).replace(tostrip, ''.encode(encoding))
clear_nullbytes(inpt, encoding)

In [ ]:


# def bytes_prntble(inpt: bytes, encoding: str = None) -> bytes:
#     ''' Same as is_printable, but for bytes in native file encoding '''
#     encoding = [encoding if encoding else get_bencod(inpt)][0]

#     b_printable = ''.encode(encoding).join([ch.encode(encoding)
#                               for ch in list(string.printable)])
#     return ''.encode(encoding).join([str(chr(int.from_bytes(ch, sys.byteorder))).encode(encoding) for ch in 
#                                     list(inpt) if str(chr(int.from_bytes(ch, sys.byteorder))).encode(encoding)
#                                     in b_printable])
    
# def get_nullrep(inpt: bytes, encoding: str = None) -> bytes:
#     ''' Returns null byte representation as bytes in native file encoding'''
#     encoding = [encoding if encoding else get_bencod(inpt)][0]
#     return bytes([inpt.splitlines(keepends = True)[-1][-1]]).decode(encoding).encode(encoding)

# def strip_null(inpt: bytes, encoding: str = None, replace_val: str = None) -> bytes:
#     ''' Remove null bytes from byte stream with proper representation
#         Adapted from:
#         https://stackoverflow.com/questions/21017698/converting-int-to-bytes-in-python-3
#         All files end by a null byte, so the last byte in a file shows
#         how null bytes are represented within this file '''
#     encoding = [encoding if encoding else get_bencod(inpt)][0]
#     repval = ['' if not replace_val else replace_val][0].encode(encoding)
#     return bytes_prntble(inpt.replace(get_nullrep(inpt, encoding), repval).replace(chr(0).encode(encoding),
#                                         repval).replace(chr(1).encode(encoding),
#                                                         repval))

# snif.get_nullrep(abyte)

In [ ]:
abyte = list(test.values())[0].iloc[1].bsheets
enc = snif.get_bencod(abyte)
# (abyte.decode(enc, 'replace').splitlines()[0], repr(abyte.decode(enc, 'replace').splitlines()[0]),
#  repr(snif.force_utf8(abyte).splitlines()[0].decode())
# )
repr(abyte.decode(enc, 'replace'))
def to_utf8(astring):
    return unidecode(repr(astring.decode(enc, 'replace').replace('�', '').encode(
              'ISO-8859-1', 'ignore').decode('ISO-8859-1', 'replace').replace('�', ''))).encode()
# len(to_utf8(abyte).splitlines()
# )
len(abyte.splitlines())
new = b'\n'.join([b'\t'.join([to_utf8(itm) for itm in to_utf8(line).split()])
            for line in abyte.splitlines()]).decode()

df([' '.join(line.split("\\x00")).split() for line in new.splitlines()])
# clean_utf8 = '\n'.join([['\t'.join([to_utf8(itm) for itm in line.split()])]
#               for line in abyte.splitlines()])
# clean_utf8
# newsheet = repr(abyte.replace(bytes(str(chr(0)), enc),
#               str(np.nan).encode(enc)).replace(
#                   bytes("'\\\\x0'", enc), str(np.nan).encode(enc)).replace(
#                   bytes("'\\\\x'", enc), ''.encode(enc)).replace(
#                       bytes(str(np.nan), enc), str("").encode(enc)).decode(
#                           'ascii', 'replace').replace('�', '').replace('ÿ', ''))

# cleaned = '\n'.join(['\t'.join([itm.replace('\\x0', ' ').strip().replace('\\x', ' ').strip() for itm in line.split()])
#            for line in repr(newsheet).replace('\\x00', ' ').splitlines()]).strip()[1:].encode().decode()
# pd.read_csv(StringIO(unidecode(cleaned)), sep='\t')
# [line.replace('\\x00', str(np.nan)).replace('\\x0', str(np.nan)) for line in newsheet.splitlines() if line != '']

In [ ]:
# abyte = list(test.values())[0].iloc[2].bsheets
# encod = list(test.values())[0].iloc[2].encoding


    
enc = snif.get_bencod(snif.strip_null(abyte))
# snif.strip_null(abyte)
# [strip_null(line) for line in strip_null(bytes_prntble(abyte)).splitlines(keepends = True)]

# get_nullrep(abyte)+b'1'
# # int.from_bytes(b'\\x0', 'little'), int.from_bytes(b'\\x0', sys.byteorder), int.from_bytes(b'\\x0', 'big')

# # newsheet = '\n'.join(['\t'.join([str(itm).replace("'", "").replace("'", '"') for itm in
# #                                  repr(line).replace('ÿ', '\s').replace('\\x0', '\s').replace(
# #                                      '\\x', '\s').replace('\s', '').split()])
# #                       for line in list(test.values())[0].iloc[5].bsheets.decode(
# #                encod, 'replace').splitlines()]).splitlines()
# # newsheet
# # strtest = abyte.splitlines()[0]

# repr(abyte.replace(repr(chr(0)).encode(encod), ''.encode(encod)).decode(encod, 'replace').replace('�', '')).encode(
#     'ISO-8859-1', 'ignore').decode('ISO-8859-1', 'replace').replace('�', '').encode()
# abyte.decode('ISO-8859-1').encode('ascii', 'ignore').decode('utf8', 'replace').replace('�', '')

# unidecode(repr(abyte.decode(encod, 'replace').replace('�', '')))
# is_printable(abyte.decode(encod, 'replace').replace('�', '').encode().replace(chr(0).encode(), ''.encode()).decode())

# # new = [[unidecode(itm).encode('ascii', 'ignore').decode('ascii', 'replace').replace('�', '')
# #         for itm in line.replace(repr(chr(0)),
# #                                '').split()
# #       if unidecode(is_printable(itm)) != '']
# #      for line in abyte.decode(encod, 'replace').replace('�', '').splitlines()
# #      if line != []]
# # new[0], ''.join(new[0]).replace(repr(chr(0)).encode(encod), ' '.encode(encod))
# new


In [ ]:
'a' in string.printable #list(string.printable)

chr(int.from_bytes(b'z', sys.byteorder))

In [ ]:
# chr(int.from_bytes(b'\\x00', 'big'))
[chr(itm).encode() for itm in list(string.printable)]
# chr(int.from_bytes('\\x0'.encode('Windows-1252'), sys.byteorder))
# chr(int.from_bytes(b'\\x00', sys.byteorder))

In [ ]:

# abyte.decode('utf8', 'replace').replace('�', '').encode().decode()
# unidecode(abyte.decode(snif.get_bencod(abyte), 'replace').replace('�', '')).encode()
snif.force_utf8(abyte, enc).splitlines()
clean1 = re.sub('\\x00', '', abyte.decode('ascii', 'replace').replace('�', '').encode('utf16').decode('utf8', 'replace'))
# unidecode(clean1.encode().decode()).encode('utf16', 'replace')
clean1
[re.sub(bytes(str(np.nan), enc), bytes('', enc),
        re.sub(bytes("\x00", enc),
               bytes(str(np.nan), enc),
               line)).decode('utf8', 'replace').replace('�', '').replace('\\', '')#unidecode(line.decode(enc, 'replace'))).encode('ascii', 'replace').decode('utf8', 'replace').replace('�', '')
  for line in abyte.splitlines()][0].split()[0]
# snif.force_utf8(abyte.decode(enc, 'replace'))
#.decode(enc, 'replace')]
# re.sub('\x0', '', clean1)
# unidecode(
#           .replace('\x00', '').replace(
#     '\\x0', '').replace('\\x', ''))
#.replace(b"\x".decode(), '')

In [ ]:
# chr(0)
# str("\xff")
chr(1).encode()

In [ ]:
bytes('n\a', 'Windows-1252')
unidecode(b'\x0b\x0c'.decode(enc))

In [ ]:
# def load_tsvs(cimaq_dir):
#     qc_ok = sorted([str(itm[0]) for itm in
#                 pd.read_csv(get_cimaq_dir_paths(
#                     cimaq_dir)[0].mean_qc.fpaths, sep='\t').values])
#     qc_ok
#     to_exclude = df(sorted([(str(bname(itm).split('_')[0]),
#                          str(bname(itm).split('_')[1]), itm) for itm in
#                         loadimages(get_cimaq_dir_paths(
#                             cimaq_dir)[0].zeprimes.fpaths)
#                         if str(bname(itm).split('_')[1]) not in qc_ok]),
#                    columns = ['pscid', 'dccid', 'fpaths'])`.set_index(
#                        'dccid').sort_index().reset_index().fpaths.tolist()

#     repair_dataset(get_cimaq_dir_paths(cimaq_dir)[0].zeprimes.fpaths,
#                    get_cimaq_dir_paths(cimaq_dir)[0].temp_events_dir.fpaths,
#                    exclude = ['pratique', 'practice', '.pdf', '.edat2'] + to_exclude)
#     allids = repair_enc_task(cimaq_dir)
#     pscids, dccids, subids = allids.pscid, allids.dccid, allids.subids
#     cimaq = pd.concat([subids, pscids, dccids] + \
#                        [loadfiles(loadimages(row[1].fpaths)).dropna(axis = 0).T
#                         for row in get_cimaq_dir_paths(cimaq_dir)[0].T.iloc[2: 4].iterrows()]).T
#     test = dict((grp, dict((sgrp, cimaq.groupby('subids').get_group(grp).groupby(
#                'fname').get_group(sgrp)) for sgrp in allscans.groupby(
#                    'subids').get_group(grp).groupby('fname').groups))
#                 for grp in allscans.groupby('subids').groups)
# #     cimaq = pd.concat([subids, pscids, dccids] + \
# #                       [loadfiles(loadimages(cimaqrow[1].fpaths)).dropna(axis = 0).rename(
# #                 columns = {'fpaths': cimaqrow[1].suffixes})#['fpaths']
# #                                   for cimaqrow in get_cimaq_dir_paths(cimaq_dir)[0].T.iloc[1: 6].iterrows()],
# #                       axis = 1).dropna(axis = 0).T.reset_index(drop = True).T
# #     cimaq = cimaq.rename(columns = {0: 'subid', 1: 'pscid', 2: 'dccid', 3: 'stereonl', 4: 'behavioral',
# #                                     5: 'confounds', 6: 'events', 7:'func'})
#     return test
# cimaq = load_tsvs(cimaq_dir)

In [ ]:
cimaq = fetch_cimaq(xpu(cimaq_dir))
#cimaq.sort_values('dccid').set_index('dccid', drop = True).reset_index()

In [ ]:
cimaq

In [ ]:
# def fetch_cimaq(cimaq_dir):
#     qc_ok = sorted([str(itm[0]) for itm in
#                 pd.read_csv(get_cimaq_dir_paths(
#                     cimaq_dir)[0].mean_qc.fpaths, sep='\t').values])
#     qc_ok
#     to_exclude = df(sorted([(str(bname(itm).split('_')[0]),
#                          str(bname(itm).split('_')[1]), itm) for itm in
#                         loadimages(get_cimaq_dir_paths(
#                             cimaq_dir)[0].zeprimes.fpaths)
#                         if str(bname(itm).split('_')[1]) not in qc_ok]),
#                    columns = ['pscid', 'dccid', 'fpaths']).set_index(
#                        'dccid').sort_index().reset_index().fpaths.tolist()

#     repair_dataset(get_cimaq_dir_paths(cimaq_dir)[0].zeprimes.fpaths,
#                    get_cimaq_dir_paths(cimaq_dir)[0].temp_events_dir.fpaths,
#                    exclude = ['pratique', 'practice', '.pdf', '.edat2'] + to_exclude)
#     allids = repair_enc_task(cimaq_dir)
#     pscids, dccids, subids = allids.pscid, allids.dccid, allids.subids
#     cimaq = pd.concat([subids, pscids, dccids] + [loadfiles(loadimages(cimaqpath)).dropna(axis = 0)['fpaths']
#                                   for cimaqpath in get_cimaq_dir_paths(cimaq_dir)[0].loc['fpaths'][2: 5]],
#                       axis = 1).dropna(axis = 0).T.reset_index(drop = True).T
#     cimaq = cimaq.rename(columns = {0: 'subid', 1: 'pscid', 2: 'dccid', 3: 'behavioral', 4: 'confounds',
#                                     5: 'events'})
#     return cimaq.set_index('dccid').sort_index()#.reset_index(drop = False)

In [ ]:
def load_scans(cimaq_dir):
    cimaq = fetch_cimaq(cimaq_dir)

#     cimaq = pd.concat([subids, pscids, dccids] + \
#                   [loadfiles(loadimages(cimaqrow[1].fpaths)).dropna(axis = 0).rename(
#             columns = {'fpaths': cimaqrow[1].suffixes})#['fpaths']
#                               for cimaqrow in get_cimaq_dir_paths(cimaq_dir)[0].T.iloc[1: 6].iterrows()],
#                   axis = 1).dropna(axis = 0).T.reset_index(drop = True).T
    
    allscans = pd.concat([loadfiles(loadimages(pjoin(dname(xpu(cimaq_dir)),
                                                     sub))).dropna(axis = 0).T
                          for sub in [itm for itm in ls(dname(xpu(cimaq_dir)))
                        if itm.startswith('sub-')]], axis = 1).T
    allscans[['dccid', 'modality', 'general']] = [(bname(row[1].fpaths).split('_')[0].split('-')[1],
                                                   bname(row[1].fpaths).split('_')[-1],
                                                   bname(dname(row[1].fpaths)))
                         for row in allscans.iterrows()]

    test = dict((grp, dict((sgrp, allscans.groupby('dccid').get_group(grp).groupby(
               'general').get_group(sgrp)) for sgrp in allscans.groupby(
                   'dccid').get_group(grp).groupby('general').groups))
                for grp in allscans.groupby('dccid').groups)
    allscans = df.from_dict(test, orient = 'index').sort_index()
    indexes = set.intersection(set(cimaq.index), set(allscans.index)) 
    return pd.concat([allscans.loc[indexes], cimaq.loc[indexes]], axis = 0)
test = load_scans(cimaq_dir)

In [ ]:
display(test)

In [ ]:
list(test2)

In [ ]:
test2.iloc[0].fmap

In [ ]:
# params = [pd.concat([row[1][['fname']], df.from_dict(json_read(row[1].fpaths, 'r'), orient = 'index')])
#           for row in test[0].iterrows()
#           if row[1].ext == '.json']
# # allparams = pd.concat(((itm.groupby('fname').get_group(grp)
# #                  for grp in itm.groupby('fname').groups)
# #              for itm in allparams), axis = 1)
# # pd.concat(params, axis =1).T.sort_values('fname')
# pd.concat(params, axis = 1).T.set_index('fname')

In [ ]:
def _update_timestamp(path: os.PathLike, set_new: bool) -> None:
    """
    Context manager to set the timestamp of the path to plus or
    minus a fixed delta, regardless of modifications within the context.

    if set_new is True, the delta is added. Otherwise, the delta is subtracted.
    """
    stats = os.stat(path)
    if set_new:
        new_timestamp = (stats.st_atime_ns + _TIMESTAMP_DELTA, stats.st_mtime_ns + _TIMESTAMP_DELTA)
    else:
        new_timestamp = (stats.st_atime_ns - _TIMESTAMP_DELTA, stats.st_mtime_ns - _TIMESTAMP_DELTA)
    try:
        yield
    finally:
        os.utime(path, ns=new_timestamp)


# Public Methods 